In [20]:
import geopandas as gpd


postcodes = gpd.read_file("../../data/raw/external/postcode/POSTCODE_POLYGON.shp")
parks = gpd.read_file("../../data/raw/external/PARKERS/PARKRES.shp")

# Check if the Coordinate Reference Systems (CRS) of both datasets match
if postcodes.crs != parks.crs:
    parks = parks.to_crs(postcodes.crs)

# spatial join
parks_in_postcode = gpd.sjoin(parks, postcodes, how="inner", predicate="within")

# count the number of parks in each postcode area
# Group by postcode and calculate the number of parks and mean area using the 'TOTAL_AREA' column
parks_stats = parks_in_postcode.groupby("POSTCODE").agg(
    park_count=("POSTCODE", "size"),
    mean_park_area=("TOTAL_AREA", "mean")
).reset_index()

print(parks_stats)
#parks_stats.to_csv("../../data/landing/parks.csv", index=False)


    POSTCODE  park_count  mean_park_area
0       3004           3       73.941667
1       3011           2       84.240000
2       3015           2      103.036000
3       3016           1       18.041000
4       3018           1        5.396000
..       ...         ...             ...
498     3988           1       22.224000
499     3991           2       26.384000
500     3992           1        4.249000
501     3995          38      500.322421
502     3996          10     1927.782400

[503 rows x 3 columns]


In [21]:
postcodes = pd.read_csv("../../data/raw/external/australian_postcodes.csv")


In [22]:
import geopandas as gpd

# 读取邮政编码和公园的shapefile
postcodes = gpd.read_file("../../data/raw/external/postcode/POSTCODE_POLYGON.shp")
parks = gpd.read_file("../../data/raw/external/PARKERS/PARKRES.shp")

# 检查两个数据集的坐标参考系统 (CRS) 是否匹配
if postcodes.crs != parks.crs:
    parks = parks.to_crs(postcodes.crs)

# 进行空间连接，使用 "intersects" 谓词，只要公园与邮政编码有部分重叠就算匹配
parks_in_postcode = gpd.sjoin(parks, postcodes, how="inner", predicate="intersects")

# 统计每个邮政编码区域内的公园数量和公园的平均面积（基于 'TOTAL_AREA' 列）
parks_stats = parks_in_postcode.groupby("POSTCODE").agg(
    park_count=("POSTCODE", "size"),  # 计算每个邮政编码的公园数量
    mean_park_area=("TOTAL_AREA", "mean")  # 计算每个邮政编码的公园平均面积
).reset_index()

# 输出结果并保存到CSV
print(parks_stats)
parks_stats.to_csv("../../data/landing/parks.csv", index=False)


    POSTCODE  park_count  mean_park_area
0       3004           4      110.079750
1       3011           2       84.240000
2       3015           3      103.036000
3       3016           5       38.051000
4       3018           1        5.396000
..       ...         ...             ...
555     3990           1      382.573000
556     3991           5      221.444200
557     3992           3     1099.514000
558     3995          43      641.789953
559     3996          14     1766.632571

[560 rows x 3 columns]


In [23]:
import geopandas as gpd
import pandas as pd

# Load parks shapefile
parks = gpd.read_file("../../data/raw/external/PARKERS/PARKRES.shp")

# Load Australian postcode CSV
postcodes = pd.read_csv("../../data/raw/external/australian_postcodes.csv")

# Convert postcodes CSV to GeoDataFrame (assuming it has latitude and longitude columns)
postcodes_gdf = gpd.GeoDataFrame(postcodes, 
                                 geometry=gpd.points_from_xy(postcodes['long'], postcodes['lat']),
                                 crs="EPSG:4326")

# Ensure both GeoDataFrames are in the same projection
parks = parks.to_crs(epsg=4326)

# Spatial join between parks and postcodes
parks_postcodes = gpd.sjoin(parks, postcodes_gdf, how="inner", op="intersects")

# Calculate the number of parks and mean park area for each postcode
parks_postcodes['park_area'] = parks_postcodes.geometry.area  # Assuming area is calculated from geometry

park_stats = parks_postcodes.groupby('postcode').agg(
    park_count=('geometry', 'size'),  # Count the number of parks
    mean_park_area=('park_area', 'mean')  # Calculate mean park area
).reset_index()

print(park_stats)
# Display the resulting DataFrame
#import ace_tools as tools; tools.display_dataframe_to_user(name="Parks and Postcodes Stats", dataframe=park_stats)


    postcode  park_count  mean_park_area
0       3004           1        0.000222
1       3090           1        0.000027
2       3231           1        0.010027
3       3234           3        0.018492
4       3235           1        0.010027
5       3237           8        0.010600
6       3239           4        0.002872
7       3250           4        0.002811
8       3251           4        0.000059
9       3294           7        0.000024
10      3314           1        0.110017
11      3340           1        0.005276
12      3352          61        0.002560
13      3364           1        0.000894
14      3374           1        0.000811
15      3377          24        0.000811
16      3381          25        0.000159
17      3385           1        0.018396
18      3409           3        0.000880
19      3424           3        0.008232
20      3441           1        0.000858
21      3458           9        0.002742
22      3465          23        0.000014
23      3477    

/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/4f/qwqd2cfs1674nsc2kmx8hhp00000gn/T/ipykernel_27792/3557847235.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_postcodes['park_area'] = parks_postcodes.geometry.area  # Assuming area is calculated from geometry


In [24]:
import geopandas as gpd
import pandas as pd

# Load parks shapefile
parks = gpd.read_file("../../data/raw/external/PARKERS/PARKRES.shp")

# Load Australian postcode CSV
postcodes = pd.read_csv("../../data/raw/external/australian_postcodes.csv")

# Convert postcodes CSV to GeoDataFrame (assuming it has latitude and longitude columns)
postcodes_gdf = gpd.GeoDataFrame(postcodes, 
                                 geometry=gpd.points_from_xy(postcodes['long'], postcodes['lat']),
                                 crs="EPSG:4326")

# Ensure both GeoDataFrames are in the same projection
parks = parks.to_crs(epsg=4326)

# Spatial join between parks and postcodes
parks_postcodes = gpd.sjoin(parks, postcodes_gdf, how="inner", op="intersects")

# Use existing park area column instead of geometry's area
# Replace 'park_area_column' with the actual column name in the shapefile that contains park area information
park_stats = parks_postcodes.groupby('postcode').agg(
    park_count=('geometry', 'size'),  # Count the number of parks
      # Calculate mean park area from the column
).reset_index()

# Display the resulting DataFrame
print(park_stats)


    postcode  park_count
0       3004           1
1       3090           1
2       3231           1
3       3234           3
4       3235           1
5       3237           8
6       3239           4
7       3250           4
8       3251           4
9       3294           7
10      3314           1
11      3340           1
12      3352          61
13      3364           1
14      3374           1
15      3377          24
16      3381          25
17      3385           1
18      3409           3
19      3424           3
20      3441           1
21      3458           9
22      3465          23
23      3477           3
24      3488           2
25      3490           2
26      3523          10
27      3533           9
28      3549           1
29      3559           1
30      3677           4
31      3699           4
32      3705           2
33      3707           1
34      3709           1
35      3737           1
36      3740           3
37      3741           7
38      3757           7


/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [25]:
# Check the columns after the spatial join
print(parks_postcodes.columns)


Index(['PRIMS_ID', 'NAME', 'AREA_TYPE', 'NAME_SHORT', 'AREA_SRC', 'MANAGER',
       'VEAC_REC', 'VEAC_STUDY', 'IUCN', 'TOTAL_AREA', 'POLY_SRC',
       'AREATYPEID', 'SRCHNAME', 'ESTAB_DATE', 'LAST_MOD', 'VERS_DATE',
       'HECTARES', 'AREASQM', 'geometry', 'index_right', 'id', 'postcode',
       'locality', 'state', 'long', 'lat', 'dc', 'type', 'status', 'sa3',
       'sa3name', 'sa4', 'sa4name', 'region', 'Lat_precise', 'Long_precise',
       'SA1_CODE_2021', 'SA1_NAME_2021', 'SA2_CODE_2021', 'SA2_NAME_2021',
       'SA3_CODE_2021', 'SA3_NAME_2021', 'SA4_CODE_2021', 'SA4_NAME_2021',
       'RA_2011', 'RA_2016', 'RA_2021', 'RA_2021_NAME', 'MMM_2015', 'MMM_2019',
       'ced', 'altitude', 'chargezone', 'phn_code', 'phn_name', 'lgaregion',
       'lgacode', 'electorate', 'electoraterating', 'sed_code', 'sed_name'],
      dtype='object')


In [26]:
parks

,PRIMS_ID,NAME,AREA_TYPE,NAME_SHORT,AREA_SRC,MANAGER,VEAC_REC,VEAC_STUDY,IUCN,TOTAL_AREA,POLY_SRC,AREATYPEID,SRCHNAME,ESTAB_DATE,LAST_MOD,VERS_DATE,HECTARES,AREASQM,geometry
0,5144,Wombat - Lerderderg National Park (Proposed),PROPOSED NATIONAL PARKS ACT PARK OR PARK ADDITION,Wombat-Lerderderg NP (Proposed),GIS PARKRES,DEECA,A4,Central West Investigation (2019),NaN,23734.951,VM PARCEL EDITED WITH 25K FEATURES,33,Wombat-Lerderderg National Park (Proposed),NaN,2024-08-07,2024-09-03,4922.15,49221504.60,"MULTIPOLYGON (((144.20860 -37.38931, 144.20864..."
1,3223,Glenmaggie Regional Park,REGIONAL PARK - NOT SCHEDULED UNDER NATIONAL P...,Glenmaggie RP,GIS PARKRES,Parks Victoria,A4,Gippsland Lakes Hinterland (1983),Not a Protected Area,570.861,HOLDING,30,GLENMAGGIE RP REGIONAL PARK R.P.,1984-06-19,2023-10-30,2024-09-03,18.80,188008.61,"MULTIPOLYGON (((146.75825 -37.92989, 146.75829..."
2,352,Yarrara Flora and Fauna Reserve,NATURE CONSERVATION RESERVE - FLORA AND FAUNA ...,Yarrara FFR,GIS PARKRES,Parks Victoria,G1,Mallee Review (1989),Ia,2267.804,HOLDING,21,YARRARA FFR FLORA AND FAUNA RESERVE F.F.R.,1979-05-01,2017-10-02,2024-09-03,1.29,12893.57,"POLYGON ((141.42818 -34.39946, 141.42734 -34.3..."
3,2746,Murray River K15 Streamside Reserve,NATURAL FEATURES RESERVE - STREAMSIDE RESERVE,Murray River K15 SSR,GIS PARKRES,Parks Victoria,H4,Box-Ironbark Investigation (2001),III,3.702,VM PARCEL EDITED WITH 25K FEATURES,18,MURRAY RIVER K15 SSR STREAMSIDE RESERVE SS.R.,1986-09-30,2024-08-08,2024-09-03,3.70,37022.24,"POLYGON ((146.61136 -36.00961, 146.61144 -36.0..."
4,5075,Nooramunga Marine and Coastal Park (addition) ...,PROPOSED NATIONAL PARKS ACT PARK OR PARK ADDITION,Nooramunga Marine and Coastal Park (addition) NHP,GIS PARKRES,DEECA,none,No LCC Recommendation,Not a Protected Area,8.633,HOLDING,33,NOORAMUNGA MARINE AND COASTAL PARK (ADDITION) ...,NaN,2018-02-07,2024-09-03,8.63,86329.22,"POLYGON ((146.79820 -38.64208, 146.79499 -38.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8782,2857,Puffing Billy Historic & Cultural Features Res.,HISTORIC RESERVE,Puffing Billy HR,GIS PARKRES,Other Government Authority,F4,Melbourne 2 Review (1994),Not a Protected Area,88.696,VM PARCEL EDITED WITH 25K FEATURES,4,PUFFING BILLY HR HISTORIC & CULTURAL FEATURES ...,1997-06-17,2017-10-02,2024-09-03,0.65,6476.31,"POLYGON ((145.50783 -37.94835, 145.50775 -37.9..."
8783,1109,Natya Bushland Reserve,NATURAL FEATURES RESERVE - BUSHLAND RESERVE,Natya BR,GIS PARKRES,Parks Victoria,I55,Mallee Review (1989),IV,30.963,VM PARCEL EDITED WITH 25K FEATURES,9,NATYA BR BUSHLAND RESERVE B.R.,1979-05-01,2016-10-10,2024-09-03,0.83,8281.31,"POLYGON ((143.22939 -34.95552, 143.22926 -34.9..."
8784,1600,Toolangi Bushland Reserve,NATURAL FEATURES RESERVE - BUSHLAND RESERVE,Toolangi BR,GIS PARKRES,Parks Victoria,G59,Melbourne 2 Review (1994),IV,30.059,VM PARCEL EDITED WITH 25K FEATURES,9,TOOLANGI BR BUSHLAND RESERVE B.R.,1979-03-20,2024-08-01,2024-09-03,12.56,125619.56,"POLYGON ((145.47150 -37.54080, 145.47150 -37.5..."
8785,1770,Crib Point G229 Bushland Reserve,NATURAL FEATURES RESERVE - BUSHLAND RESERVE,Crib Point G229 BR,GIS PARKRES,Committee of Management,G229,Melbourne 2 Review (1994),IV,0.693,VM PARCEL EDITED WITH 25K FEATURES,9,CRIB POINT G229 BR BUSHLAND RESERVE B.R.,1997-06-17,2023-07-12,2024-09-03,0.69,6926.96,"POLYGON ((145.20118 -38.36090, 145.20120 -38.3..."


In [27]:
max_park_postcode = parks_stats.loc[parks_stats['park_count'].idxmax()]

# most parks postcode
print(f"postcide {max_park_postcode['POSTCODE']} with max {max_park_postcode['park_count']} parks")

postcide 3551 with max 240 parks
